In [1]:
import pandas as pd

## Veld gewassen

In [12]:
df_raw = pd.read_excel('data/Teeltplan NoordOogst 2025.xlsx', sheet_name='Veld gewassen')

In [13]:
df_raw.columns

Index(['Groep', 'Gewas', 'Ras', 'Aantal bedden', 'Z / VZ / PL ',
       'Afstand tussen rij', 'Afstand in rij', 'Aantal rijen',
       'Planten per bed', 'Planten nodig', 'Zaaidiepte (cm)', 'Zaaien (week)',
       'Planten (week)', 'Verwachte oogst (week)', 'Eindweek\noogst',
       'Einddatum oogst', 'Bakken', 'Aantal planten besteld',
       'Duizendkorrel gewicht', 'Zaad nodig (gr)', 'Leverancier',
       'Opmerkingen'],
      dtype='object')

In [14]:
def get_unique_bed_numbers(df):
    cleaned = df['Bed\nnummer'].str.replace(" ","").str.split(',').dropna()
    return {e for l in cleaned for e in l}

data = list()
mapper = {
    'zaaien': 'Zaaien (week)',
    'planten': 'Planten (week)'} 
for week in range(2, 53):
    for activity in mapper.keys():
        selection = df_raw[df_raw[mapper[activity]] == week]
        for gewas in set(selection['Gewas']):
            this_rows = selection[selection['Gewas'] == gewas]
            if len(this_rows['Z / VZ / PL '].unique()) > 2:
                raise ValueError(f'Multiple methods for {gewas}')
            method = this_rows['Z / VZ / PL '].unique()[0]
            if activity in ['zaaien', 'planten'] and method == 'Z':
                to_do = 'direct zaaien'
            elif activity == 'zaaien' and method == 'VZ':
                to_do = 'voorzaaien'
                bed_numbers = []
            else:
                to_do = activity

                
            data.append({'Week': week,
                         'Wat doen?': f'{gewas} {to_do}'})
veld_per_week_df = pd.DataFrame(data)

In [15]:
veld_per_week_df.head()

,Week,Wat doen?
0,4,Tuinbonen (vroeg) voorzaaien
1,8,Peultjes voorzaaien
2,8,Doperwten voorzaaien
3,8,Verbena voorzaaien
4,8,Kapucijners voorzaaien


## Kas gewassen

In [16]:
df_raw = pd.read_excel('data/Teeltplan NoordOogst 2025.xlsx', sheet_name='Kas gewassen')

In [17]:
df_raw.columns

Index(['Gewas', 'Ras', 'Aantal bedden', 'Z / VZ / PL', 'Afstand tussen rij',
       'Afstand in rij', 'Aantal rijen', 'Planten per bed', 'Planten nodig',
       'Zaaidiepte', 'Zaaien (week)', 'Planten (week)',
       'Verwachte oogst (week)', 'Eindweek\noogst', 'Bakken',
       'Aantal planten besteld', 'Duizendkorrel gewicht', 'Zaad nodig (gr)',
       'Leverancier', 'Opmerkingen'],
      dtype='object')

In [18]:
def get_unique_bed_numbers(df):
    cleaned = df['Bed\nnummer'].str.replace(" ","").str.split(',').dropna()
    return {e for l in cleaned for e in l}

data = list()
mapper = {
    'zaaien': 'Zaaien (week)',
    'planten': 'Planten (week)'} 
for week in range(2, 53):
    for activity in mapper.keys():
        selection = df_raw[df_raw[mapper[activity]] == week]
        for gewas in set(selection['Gewas']):
            this_rows = selection[selection['Gewas'] == gewas]
            if len(this_rows['Z / VZ / PL'].unique()) > 2:
                raise ValueError(f'Multiple methods for {gewas}')
            method = this_rows['Z / VZ / PL'].unique()[0]
            if activity in ['zaaien', 'planten'] and method == 'Z':
                to_do = 'direct zaaien'
            elif activity == 'zaaien' and method == 'VZ':
                to_do = 'voorzaaien'
                bed_numbers = []
            else:
                to_do = activity

                
            data.append({'Week': week,
                         'Wat doen?': f'{gewas} {to_do}'})
kas_per_week_df = pd.DataFrame(data)

In [19]:
kas_per_week_df.head()

,Week,Wat doen?
0,7,paprika eigen opkweek voorzaaien
1,7,spinazie direct zaaien
2,7,peper eigen opkweek geleid voorzaaien
3,7,bosui direct zaaien
4,7,peper eigen opkweek bush voorzaaien


In [20]:
kas_per_week_df.shape

(48, 2)

In [22]:
full_df = pd.concat([veld_per_week_df, kas_per_week_df])

In [23]:
full_df = full_df.sort_values(by='Week')

In [24]:
full_df.head()

,Week,Wat doen?
0,4,Tuinbonen (vroeg) voorzaaien
2,7,peper eigen opkweek geleid voorzaaien
3,7,bosui direct zaaien
4,7,peper eigen opkweek bush voorzaaien
1,7,spinazie direct zaaien


In [25]:
full_df.to_excel('data/planning-per-week-noordoogst-2025.xlsx', index=False)